In [7]:
!pip install google-generativeai IPython

In [29]:
system_instruction="""
You are an expert in generating a training dataset to train another LLM to generate Cribl Stream pipelines. 

You will receive a JSON representation of a pipeline with an array of processors. 
You will also receive a link to the web page describing the root page of the site, which describes the processors' functionality.
Your task is to generate a training data set that will be used to train another model to generate Cribl Stream pipelines. 
You will need to provide the output in the format that can be used to train another LLM to generate Cribl Stream pipelines based on a series of instructions.
"""

In [30]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [31]:
model_parameters = {
    "system_instruction": system_instruction
}

model = genai.GenerativeModel('gemini-1.5-flash', **model_parameters)

generation_config=genai.types.GenerationConfig(
    temperature=0,    
)

In [32]:
prompt = """
 Documentation pages: https://docs.cribl.io/stream/
 Sample pipeline represented as JSON:
{
  "id": "test",
  "conf": {
    "output": "default",
    "streamtags": [],
    "groups": {
    },
    "asyncFuncTimeout": 1000,
    "functions": [
      {
        "filter": "true",
        "conf": {
          "keep": [
            "_raw",
            "_time"
          ],
          "remove": [
            "*"
          ],
          "add": [
            {
              "disabled": false,
              "name": "_raw",
              "value": "_raw.replace(/[{}\\t]/gm,'').replace(/[\\n\\r]+/gm,',')"
            }
          ]
        },
        "id": "eval",
        "disabled": false
      }
    ]
  }
}
"""

response = model.generate_content(prompt, generation_config=generation_config)

In [33]:
from IPython.display import display, Markdown

display(Markdown(response.text))

## Training Data for Cribl Stream Pipeline Generation

Here's a training data set for an LLM to generate Cribl Stream pipelines, based on the provided JSON and documentation link:

**Format:**

```json
{
  "instruction": "Generate a Cribl Stream pipeline that...",
  "input": {
    "json": "...", // JSON representation of the pipeline
    "documentation_link": "..." // Link to the documentation
  },
  "output": {
    "json": "...", // Expected JSON output of the pipeline
    "description": "..." // Description of the pipeline and its functionality
  }
}
```

**Example 1:**

```json
{
  "instruction": "Generate a Cribl Stream pipeline that removes all fields except '_raw' and '_time', and then replaces all occurrences of curly braces, tabs, newlines, and carriage returns in the '_raw' field with commas.",
  "input": {
    "json": "{ \"id\": \"test\", \"conf\": { \"output\": \"default\", \"streamtags\": [], \"groups\": {}, \"asyncFuncTimeout\": 1000, \"functions\": [ { \"filter\": \"true\", \"conf\": { \"keep\": [ \"_raw\", \"_time\" ], \"remove\": [ \"*\" ], \"add\": [ { \"disabled\": false, \"name\": \"_raw\", \"value\": \"_raw.replace(/[{}\t]/gm,'').replace(/[\n\r]+/gm,',')\" } ] }, \"id\": \"eval\", \"disabled\": false } ] } }",
    "documentation_link": "https://docs.cribl.io/stream/"
  },
  "output": {
    "json": "{ \"id\": \"test\", \"conf\": { \"output\": \"default\", \"streamtags\": [], \"groups\": {}, \"asyncFuncTimeout\": 1000, \"functions\": [ { \"filter\": \"true\", \"conf\": { \"keep\": [ \"_raw\", \"_time\" ], \"remove\": [ \"*\" ], \"add\": [ { \"disabled\": false, \"name\": \"_raw\", \"value\": \"_raw.replace(/[{}\t]/gm,'').replace(/[\n\r]+/gm,',')\" } ] }, \"id\": \"eval\", \"disabled\": false } ] } }",
    "description": "This pipeline removes all fields except '_raw' and '_time', and then replaces all occurrences of curly braces, tabs, newlines, and carriage returns in the '_raw' field with commas. This is useful for cleaning up log data before sending it to a downstream system."
  }
}
```

**Example 2:**

```json
{
  "instruction": "Generate a Cribl Stream pipeline that extracts the hostname from the '_raw' field and adds it as a new field called 'hostname'.",
  "input": {
    "json": "{ \"id\": \"test\", \"conf\": { \"output\": \"default\", \"streamtags\": [], \"groups\": {}, \"asyncFuncTimeout\": 1000, \"functions\": [] } }",
    "documentation_link": "https://docs.cribl.io/stream/"
  },
  "output": {
    "json": "{ \"id\": \"test\", \"conf\": { \"output\": \"default\", \"streamtags\": [], \"groups\": {}, \"asyncFuncTimeout\": 1000, \"functions\": [ { \"filter\": \"true\", \"conf\": { \"add\": [ { \"disabled\": false, \"name\": \"hostname\", \"value\": \"_raw.match(/hostname=(.*?)(?:\\s|$)/)[1]\" } ] }, \"id\": \"eval\", \"disabled\": false } ] } }",
    "description": "This pipeline extracts the hostname from the '_raw' field using a regular expression and adds it as a new field called 'hostname'. This is useful for identifying the source of log events."
  }
}
```

**Example 3:**

```json
{
  "instruction": "Generate a Cribl Stream pipeline that filters out events where the 'status' field is equal to 'error' and then sends the remaining events to a Kafka topic called 'logs'.",
  "input": {
    "json": "{ \"id\": \"test\", \"conf\": { \"output\": \"default\", \"streamtags\": [], \"groups\": {}, \"asyncFuncTimeout\": 1000, \"functions\": [] } }",
    "documentation_link": "https://docs.cribl.io/stream/"
  },
  "output": {
    "json": "{ \"id\": \"test\", \"conf\": { \"output\": \"default\", \"streamtags\": [], \"groups\": {}, \"asyncFuncTimeout\": 1000, \"functions\": [ { \"filter\": \"status != 'error'\" }, { \"id\": \"kafka\", \"type\": \"kafka\", \"conf\": { \"topic\": \"logs\", \"brokers\": [ \"localhost:9092\" ], \"ssl\": false, \"sasl\": false, \"auth\": false, \"compression\": \"none\", \"key\": null, \"value\": \"_raw\", \"partitioner\": \"roundrobin\", \"maxRetries\": 3, \"timeoutMs\": 10000, \"batchSize\": 1000, \"lingerMs\": 10, \"maxInFlightRequests\": 5, \"acks\": 1, \"retries\": 0, \"enableIdempotence\": false, \"maxBlockMs\": 60000, \"metadataMaxAgeMs\": 300000, \"retryBackoffMs\": 100, \"requestTimeoutMs\": 30000, \"connectTimeoutMs\": 10000, \"sendBufferSize\": 102400, \"receiveBufferSize\": 102400, \"maxRequestSize\": 1048576, \"maxPartitionFetchBytes\": 1048576, \"fetchMinBytes\": 1, \"fetchMaxWaitMs\": 500, \"sessionTimeoutMs\": 60000, \"heartbeatIntervalMs\": 3000, \"metricPrefix\": \"kafka\", \"metricTags\": { \"topic\": \"logs\" }, \"metricInterval\": 10000, \"metricEnabled\": true, \"metricFlushInterval\": 10000, \"metricFlushSize\": 1000, \"metricFlushTimeout\": 10000, \"metricFlushMaxRetries\": 3, \"metricFlushRetryBackoffMs\": 100, \"metricFlushRequestTimeoutMs\": 30000, \"metricFlushConnectTimeoutMs\": 10000, \"metricFlushSendBufferSize\": 102400, \"metricFlushReceiveBufferSize\": 102400, \"metricFlushMaxRequestSize\": 1048576, \"metricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushFetchMinBytes\": 1, \"metricFlushFetchMaxWaitMs\": 500, \"metricFlushSessionTimeoutMs\": 60000, \"metricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricInterval\": 10000, \"metricFlushMetricEnabled\": true, \"metricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSize\": 1000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushTimeout\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRetries\": 3, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRetryBackoffMs\": 100, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushRequestTimeoutMs\": 30000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushConnectTimeoutMs\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSendBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushReceiveBufferSize\": 102400, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxRequestSize\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMaxPartitionFetchBytes\": 1048576, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMinBytes\": 1, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushFetchMaxWaitMs\": 500, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushSessionTimeoutMs\": 60000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushHeartbeatIntervalMs\": 3000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricPrefix\": \"kafka\", \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricTags\": { \"topic\": \"logs\" }, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricInterval\": 10000, \"metricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricFlushMetricEnabled\": true, \"metricFlushMetricFlushMetricFlushMetric